In [1]:
import mysql.connector  as connection
import pandas as pd
import numpy as np

In [2]:
# Creation of directory variables

files_dir = 'G:/Reports/043_Bestseller/Files/'
sql_dir = 'G:/Reports/043_Bestseller/SQL/'

In [28]:
# Fetching data from the database

with open(files_dir + 'db_credentials.txt', 'r') as f:
    db_credentials = f.read().splitlines()

try:
    db_connections = connection.connect(
        host=db_credentials[0],
        user=db_credentials[1],
        password=db_credentials[2],
        database=db_credentials[3]
    )
        # loading sales_ratio
    print("loading sales_ratio ...")
    sales_ratio_NEW = open(sql_dir + '06_sales ratio/sales ratio NEW.sql')
    sales_ratio_NEW_query = sales_ratio_NEW.read()
    sales_ratio = pd.read_sql(sales_ratio_NEW_query,db_connections)
    print('sales_ratio is ready')

    # close the connection
    db_connections.close()  
    print('MySQL query is loaded and db_connections is closed ')
except Exception as e:
    db_connections.close()
    print(str(e))

loading sales_ratio ...


C:\Users\User\AppData\Local\Temp\ipykernel_12028\1742810371.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_ratio = pd.read_sql(sales_ratio_NEW_query,db_connections)


sales_ratio is ready
All MySQL queries are loaded and db_connections is closed 


In [29]:
# Fetching data from the database

try:
    db_connections = connection.connect(
        host=db_credentials[0],
        user=db_credentials[1],
        password=db_credentials[2],
        database=db_credentials[3]
    )
        # loading article_stock
    print("loading article_stock ...")
    article_stock_NEW = open(sql_dir + '01_Article stock/Article Stock NEW.sql')
    article_stock_NEW_query = article_stock_NEW.read()
    article_stock = pd.read_sql(article_stock_NEW_query,db_connections)
    print('article_stock is ready')

    # close the connection
    db_connections.close()  
    print('MySQL query is loaded and db_connections is closed ')
except Exception as e:
    db_connections.close()
    print(str(e))

loading article_stock ...


C:\Users\User\AppData\Local\Temp\ipykernel_12028\124249812.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  article_stock = pd.read_sql(article_stock_NEW_query,db_connections)


article_stock is ready
All MySQL queries are loaded and db_connections is closed 


In [38]:
# Joining dataframes

merged_df = pd.merge(sales_ratio, article_stock, on='Article ID', how='outer')

In [39]:
# deleting dataframes which won't be used to save memory space

del sales_ratio
del article_stock

In [42]:
# Data transfrmations

merged_df['Available stock (for understock)'] = merged_df.Stock - merged_df.Reserved
merged_df.dropna(how='any', subset='SA ID', inplace=True)
merged_df = merged_df.astype(str)
merged_df[['Auftrag number', 'Main auftrag','SA ID','Total no of articles','Article Group ID']] = merged_df[['Auftrag number', 'Main auftrag','SA ID','Total no of articles','Article Group ID']].replace(r'\.0', '', regex=True)
merged_df.replace( 'nan', '', inplace=True)
merged_df.replace( 'None', '', inplace=True)
merged_df.replace( 'NaT', '', inplace=True)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [50]:
# Save the result into csv

merged_df.to_csv(files_dir + 'sales_ratio_merged.csv', index=False, encoding='utf-8', mode='w')